# Use the model in a inference behaviour

1. Load a cv config with all experiment parameters
2. Load the corresponding data, 
3. create a train and validation generator with the given parameters, exclusive the augmentation parameters
4. reconstruct the model with the given parameters, (we have custom loss functions, simple model.load() will not work)
5. load and apply the corresponding weights (with respect to the distributed training strategy)
6. predict the targt vectors with the train and val generators (make sure that we change the batchsize to 1, and avoid shuffle so that we get all files)
7. write the gt and predictions as numpy into the corresponding experiment folder

In [2]:
#checks_out

# ------------------------------------------define logging and working directory
from ProjectRoot import change_wd_to_project_root
change_wd_to_project_root()
from src.utils.Tensorflow_helper import choose_gpu_by_id
# ------------------------------------------define GPU id/s to use
GPU_IDS = '0,1'
GPUS = choose_gpu_by_id(GPU_IDS)
print(GPUS)
# ------------------------------------------jupyter magic config
%matplotlib inline
%reload_ext autoreload
%autoreload 2
# ------------------------------------------ import helpers
# this should import glob, os, and some other standard libs to keep this cell clean
# local imports
from src.utils.Notebook_imports import *
from src.utils.Utils_io import Console_and_file_logger, init_config

# import external libs
from tensorflow.python.client import device_lib
import tensorflow as tf
tf.get_logger().setLevel('ERROR')
import cv2
import pandas as pd
import numpy as np
import SimpleITK as sitk
from ipyfilechooser import FileChooser
#import nrrd as nrrd # https://pypi.org/project/pynrrd/
from src.data.Generators import DataGenerator


search for root_dir and set working directory
Working directory set to: /mnt/ssd/git/wft21_septum_landmark_detection
['/gpu:0', '/gpu:1']


# Load a config into the global namespace

In [3]:
#checks_out

exp_config_chooser = FileChooser(os.path.join(os.getcwd(),'exp/temp'), '')
display(exp_config_chooser)
@interact_manual
def load_config():

    global exp_config_chooser, config
    """
    load an experiment config
    """
    if 'exp_config_chooser' in globals():
        config_file  = exp_config_chooser.selected
    else:
        print('no config chooser found')

    # load the experiment config
    with open(config_file, encoding='utf-8') as data_file:
        config = json.loads(data_file.read())
    globals().update(config)
    Console_and_file_logger(EXPERIMENT, logging.INFO)
    logging.info('Loaded config for experiment: {}'.format(config['EXPERIMENT']))

FileChooser(path='/mnt/ssd/git/wft21_septum_landmark_detection/exp/temp', filename='', title='HTML(value='', l…

interactive(children=(Button(description='Run Interact', style=ButtonStyle()), Output()), _dom_classes=('widge…

In [4]:
df = pd.read_csv(DF_FOLDS)
df.head() # shows dataframe with all files

NameError: name 'DF_FOLDS' is not defined

In [ ]:
# # useful for troubleshooting , showcases entire dataframe
# pd.set_option("display.max_rows", None, "display.max_columns", None) 
# print(df)

In [5]:
df = df[df['fold']==3] # load dataframe with all patients and corresponding fold (in our case only fold0)
df.head() # shows dataframe with all files for fold 0

NameError: name 'df' is not defined

In [6]:
#df_test = df[df['modality']==train]

In [7]:
#for p in df['patient'].unique(): # show only data on 'unique' patients to sum up folds and slices
for p in sorted(df['patient'].unique()):
    print(p)
    files_ = df[df['patient'] ==p]['x_path'].values
    print(len(files_)) # shows amount of slices for each patient
    #print(files_[0]) # files is a list with the name of all patient data
    ed_f = files_[:len(files_)//2]
    es_f = files_[len(files_)//2:]
    print('length of ed_f ' + str(len(ed_f)))
    print('length of es_f ' + str(len(es_f)))
    #print('this is ed_f ' + ed_f)
    #print('this is es_f ' + es_f)
    

patient001
20
length of ed_f 10
length of es_f 10
patient002
20
length of ed_f 10
length of es_f 10
patient003
20
length of ed_f 10
length of es_f 10
patient004
20
length of ed_f 10
length of es_f 10
patient005
20
length of ed_f 10
length of es_f 10
patient006
22
length of ed_f 11
length of es_f 11
patient007
20
length of ed_f 10
length of es_f 10
patient008
20
length of ed_f 10
length of es_f 10
patient009
20
length of ed_f 10
length of es_f 10
patient010
20
length of ed_f 10
length of es_f 10
patient011
18
length of ed_f 9
length of es_f 9
patient012
20
length of ed_f 10
length of es_f 10
patient013
20
length of ed_f 10
length of es_f 10
patient014
20
length of ed_f 10
length of es_f 10
patient015
18
length of ed_f 9
length of es_f 9
patient016
20
length of ed_f 10
length of es_f 10
patient017
18
length of ed_f 9
length of es_f 9
patient018
16
length of ed_f 8
length of es_f 8
patient019
22
length of ed_f 11
length of es_f 11
patient020
16
length of ed_f 8
length of es_f 8
patient021

In [8]:
# ed_f = files_[:len(files_)//2]
# es_f = files_[len(files_)//2:]
# print(len(ed_f))
# print(len(es_f))

# Pandas Dataframe

# Load the corresponding file names for this fold

In [9]:
#checks_out
# Load SAX volumes
from src.data.Dataset import get_trainings_files
x_train_sax, y_train_sax, x_val_sax, y_val_sax = get_trainings_files(data_path=DATA_PATH_SAX,
                                                                     path_to_folds_df=DF_FOLDS,
                                                                     fold=FOLD)
logging.info('SAX train CMR: {}, SAX train masks: {}'.format(len(x_train_sax), len(y_train_sax)))
logging.info('SAX val CMR: {}, SAX val masks: {}'.format(len(x_val_sax), len(y_val_sax)))

2021-07-06 16:35:47,083 INFO Found 1902 images/masks in /mnt/ssd/data/WFT_MRT21/Export_2021-05-26_16_03
2021-07-06 16:35:47,084 INFO Patients train: 75
2021-07-06 16:35:47,139 INFO Selected 1406 of 1902 files with 75 of 100 patients for training fold 3
2021-07-06 16:35:47,140 INFO SAX train CMR: 1406, SAX train masks: 1406
2021-07-06 16:35:47,140 INFO SAX val CMR: 496, SAX val masks: 496


# Create the same the train and val generators as used for the training of this model

Make sure that:
- no shuffle
- no augmentation (classic and temporal)
- batchsize equal 1

In [10]:
#checks_out
# DataGenerator for SAX, changed from PhaseRegression Generator
# logging.getLogger().setLevel(logging.INFO)
from src.data.Generators import DataGenerator
config['SHUFFLE'] = False
config['AUGMENT'] = False
config['AUGMENT_GRID'] = False# make sure no augmentation will be applied to the validation data
config['HIST_MATCHING'] = False
config['BATCHSIZE'] = 1
batch_generator = DataGenerator(x_train_sax, y_train_sax, config=config)
# create another config for the validation data, 
# by this we can have a different set of parameters for both generators
val_config = config.copy()
validation_generator = DataGenerator(x_val_sax, y_val_sax, config=val_config) 

2021-07-06 16:35:47,933 INFO Create DataGenerator
2021-07-06 16:35:47,939 INFO Datagenerator created with: 
 shape: [128, 128]
 spacing: [1.8, 1.8]
 batchsize: 1
 Scaler: MinMax
 Images: 1406 
 Augment: False 
 Thread workers: 8
2021-07-06 16:35:47,939 INFO No augmentation
2021-07-06 16:35:47,940 INFO Create DataGenerator
2021-07-06 16:35:47,942 INFO Datagenerator created with: 
 shape: [128, 128]
 spacing: [1.8, 1.8]
 batchsize: 1
 Scaler: MinMax
 Images: 496 
 Augment: False 
 Thread workers: 8
2021-07-06 16:35:47,943 INFO No augmentation


In [11]:
#x_val_sax

In [12]:
# print(validation_generator)

# Load the model, load and set the corresponding weights

In [13]:
#checks_out
# Create Neural Network for SAX as U-Net
import src.models.Unets as modelmanager
# create a model
logging.info('Create model')
model = modelmanager.create_unet(config)
model.load_weights(os.path.join(config['MODEL_PATH'],'model.h5'))
logging.info('loaded model weights as h5 file')
model.summary()

2021-07-06 16:35:49,909 INFO Create model


using tensorflow, need to monkey patch
tf.python.backend.slice overwritten by monkey patch
(None, 128, 128, 1)


2021-07-06 16:35:51,507 INFO loaded model weights as h5 file


Model: "unet"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 128, 128, 1) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 128, 128, 32) 320         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 128, 128, 32) 128         conv2d[0][0]                     
__________________________________________________________________________________________________
dropout (Dropout)               (None, 128, 128, 32) 0           batch_normalization[0][0]        
_______________________________________________________________________________________________

# Predict on the validation split

In [1]:
# predict on the validation generator
preds = model.predict(validation_generator)
logging.info(preds.shape)


NameError: name 'model' is not defined

In [16]:
#print(preds)

## Alternative: Predict on the training split

### predicting on both needs two seperate runs of the notebook and adjustment of some code

In [17]:
# # predict on the training generator
# preds = model.predict(batch_generator)
# logging.info(preds.shape)

# Create one numpy object from it

In [18]:
# get all ground truth vectors
gts = np.stack([np.squeeze(y) for x, y in validation_generator])
logging.info(gts.shape)

2021-07-06 16:21:23,284 INFO (476, 128, 128, 2)


In [19]:
# get all original cmr images
gts_cmr = np.stack([np.squeeze(x) for x, y in validation_generator])
logging.info(gts_cmr.shape)

2021-07-06 16:21:24,766 INFO (476, 128, 128)


# Save gt and pred into experiment folder
saves groundtruth based on np.stack with gts and predictions as one numpy stack under the pred_filename as a .npy file. The numpy stack is later used to create the images based on the arrays


In [20]:
#checks_out
pred_path = os.path.join(config['EXP_PATH'], 'predictions_'+ str(datetime.datetime.now().strftime("%Y-%m-%d_%H_%M")))
ensure_dir(pred_path)
pred_filename = os.path.join(pred_path, 'gtpred_fold{}.npy'.format(config['FOLD'])) #names file according to fold, in our case only fold0
np.save(pred_filename, np.stack([gts, preds], axis=0)) 
logging.info('saved as: \n{} \ndone!'.format(pred_filename))

2021-07-06 16:21:26,122 INFO saved as: 
exp/temp/TeaMRT_Exp1/2021-07-06_13_48/predictions_2021-07-06_16_21/gtpred_fold0.npy 
done!


In [21]:
#checks_out

#np.load('/mnt/ssd/git/wft21_septum_landmark_detection/{}'.format(pred_filename))

stack = np.load('/mnt/ssd/git/wft21_septum_landmark_detection/{}'.format(pred_filename)) # loads current predictions from folder
print('the shape of the numpy.stack is' + str(np.shape(stack)))
print(('loading from: /mnt/ssd/git/wft21_septum_landmark_detection/{}'.format(pred_filename)))

#for-loop over stack --> reading numpy and transferring with pynrrd. pad_and_crop function applied. saving with the name of original file.

the shape of the numpy.stack is(2, 476, 128, 128, 2)
loading from: /mnt/ssd/git/wft21_septum_landmark_detection/exp/temp/TeaMRT_Exp1/2021-07-06_13_48/predictions_2021-07-06_16_21/gtpred_fold0.npy


## Create Images from Arrays

One-hot encoding of annotations. Channel 1 is transformed to pixels with the value of 1, Channel 2 is transformed to pixels with the value of 2. This is the same format to annotations created as .nrrd-files with the MITK-Toolkit. 

Both the predictions and groundtruths are recreated using this method. This is the simplest way for us to scale the predictions to the same size of the groundtruth images, without needing implement undoing of the generator steps and dealing with differenzt sizing or spacing issues. 

Using the sitk toolkit a total of three images are created per training file. One representing the file without any annotations, one with our groundtruth and one with the prediction of the model.

The above created numpy stack is used for this

In [24]:
#checks_out

gts = stack[0]
preds = stack[1]


# upper = 1, lower == 2
# transform to int representation (one-hot)
gts_flat = np.zeros((gts.shape[:-1]))
gts_flat[gts[...,0]>0.5] = 1
gts_flat[gts[...,1]>0.5] = 2

preds_flat = np.zeros((gts.shape[:-1]))
preds_flat[preds[...,0]>0.5] = 1
preds_flat[preds[...,1]>0.5] = 2
print(preds_flat.shape)
#print(preds_flat) #NotIncluded
gt_sitks = [sitk.GetImageFromArray(elem) for elem in gts_flat]
pred_sitks = [sitk.GetImageFromArray(elem) for elem in preds_flat]

(476, 128, 128)


In [25]:
# save preds_flat to understand what they look like

pred_flat_path = os.path.join(config['EXP_PATH'], 'predictions_flat'+ str(datetime.datetime.now().strftime("%Y-%m-%d_%H_%M")))
ensure_dir(pred_flat_path)
pred_flat_filename = os.path.join(pred_flat_path, 'gtpred_fold{}.npy'.format(config['FOLD'])) #names file according to fold, in our case only fold0


np.save(pred_flat_filename, np.stack(preds_flat[:10], axis=0)) # saves a numpy stack of first 10 files only???

logging.info('saved as: \n{} \ndone!'.format(pred_flat_filename))


2021-07-06 11:24:43,928 INFO saved as: 
exp/temp/spacing_1_8/2021-05-26_13_40/predictions_flat2021-07-06_11_24/gtpred_fold0.npy 
done!


In [32]:
#print(pred_filename)

the sitk.WriteImage functions all use numpy stacks. This is to join corresponding images of the same 3d-volume to one image. 

Theoretically I just care about the numbers of the preds... 

In [26]:

# write 3d_nrrd file of groundtruth to disk

sitk.WriteImage(sitk.GetImageFromArray(np.stack(gts_flat[:10], axis=0)), '/mnt/ssd/git/wft21_septum_landmark_detection/data/temp/3d_new_temp_gt.nrrd')

In [29]:
# write 3d_nrrd file of predictions to disk
sitk.WriteImage(sitk.GetImageFromArray(np.stack(preds_flat[:10], axis=0)), '/mnt/ssd/git/wft21_septum_landmark_detection/data/temp/3d_new_temp_pred.nrrd')

In [28]:
# write 3d_nrrd file of original image to disk

sitk.WriteImage(sitk.GetImageFromArray(np.stack(gts_cmr[:10], axis=0)), '/mnt/ssd/git/wft21_septum_landmark_detection/data/temp/3d_new_temp_cmr.nrrd')

# Experimental Combiner

In [8]:

# # write 3d_nrrd file to disk
# #for p in df['patient'].unique(): # show only data on 'unique' patients to sum up folds and slices
# filecounter = 0
# for p in sorted(df['patient'].unique()):
#     print(p)
#     files_ = df[df['patient'] ==p]['x_path'].values
#     print(len(files_)) # shows amount of slices for each patient
#     #print(files_[0]) # files is a list with the name of all patient data
#     ed_f = files_[:len(files_)//2]
#     es_f = files_[len(files_)//2:]
#     print('length of ed_f ' + str(len(ed_f)))
#     print('length of es_f ' + str(len(es_f)))
#     #print('this is ed_f ' + ed_f)
#     #print('this is es_f ' + es_f)
#     print('filecounter starts at at: ' + str(filecounter))
#     limit = filecounter + (len(files_)//2)
#     print('limit starts at: ' + str(limit))
    
    
#     print('WRITING from :' + str(filecounter) + ' to ' + str(limit))
#     #sitk.WriteImage(sitk.GetImageFromArray(np.stack(gts_flat[:10], axis=0)), '/mnt/ssd/git/wft21_septum_landmark_detection/data/temp/3d_new_temp_gt.nrrd')
#     #sitk.WriteImage(sitk.GetImageFromArray(np.stack(preds_flat[filecounter:len(files_)//2], axis=0)), '/mnt/ssd/git/wft21_septum_landmark_detection/data/temp/ preds{}_ed.nrrd'.format(str(p))
#     sitk.WriteImage(sitk.GetImageFromArray(np.stack(preds_flat[filecounter:limit], axis=0)), '/mnt/ssd/git/wft21_septum_landmark_detection/data/temp/preds_{}_ed.nrrd'.format(str(p)))
#     #sitk.WriteImage(sitk.GetImageFromArray(np.stack(gts_cmr[:10], axis=0)), '/mnt/ssd/git/wft21_septum_landmark_detection/data/temp/3d_new_temp_cmr.nrrd')
    
#     #sitk.WriteImage(sitk.GetImageFromArray(np.stack(preds_flat[11:20], axis=0)), '/mnt/ssd/git/wft21_septum_landmark_detection/data/temp/checkpred.nrrd')
    
    
#     filecounter = filecounter + (len(files_)//2)
#     limit = filecounter + (len(files_)//2)
#     print('first round done. filecounter at: ' + str(filecounter) + ', limit at ' + str(limit))
    
    
#     print('WRITING from :' + str(filecounter+1) + ' to ' + str(limit))
#     sitk.WriteImage(sitk.GetImageFromArray(np.stack(preds_flat[filecounter:limit], axis=0)), '/mnt/ssd/git/wft21_septum_landmark_detection/data/temp/preds_{}_es.nrrd'.format(str(p)))
#     #sitk.WriteImage(sitk.GetImageFromArray(np.stack(gts_flat[:10], axis=0)), '/mnt/ssd/git/wft21_septum_landmark_detection/data/temp/3d_new_temp_gt.nrrd')
#     #sitk.WriteImage(sitk.GetImageFromArray(np.stack(preds_flat[filecounter:len(files_)//2], axis=0)), '/mnt/ssd/git/wft21_septum_landmark_detection/data/temp/preds{}_ed.nrrd'.format(p))
#     #sitk.WriteImage(sitk.GetImageFromArray(np.stack(gts_cmr[:10], axis=0)), '/mnt/ssd/git/wft21_septum_landmark_detection/data/temp/3d_new_temp_cmr.nrrd')
    
#     filecounter = filecounter + (len(files_)//2)
#     limit = 0
#     print('patient done. filecounter exits at: ' + str(filecounter) + ', limit will be reinitialized in next pass.')
    
#     print('Done')


# fast hack which aligns the patients

In [14]:
# get a list of patients for fast testing
patients = [p for p in sorted(df['patient'].unique())]
#patients

In [16]:
# in short: create one generator per patient
# this is a fast hack, it is not very fast
# DataGenerator for SAX, changed from PhaseRegression Generator
# logging.getLogger().setLevel(logging.INFO)
from src.data.Generators import DataGenerator
from logging import info
config['SHUFFLE'] = False
config['AUGMENT'] = False
config['AUGMENT_GRID'] = False# make sure no augmentation will be applied to the validation data
config['HIST_MATCHING'] = False
config['BATCHSIZE'] = 1
# by this we can have a different set of parameters for both generators
val_config = config.copy()
export_root = '/mnt/ssd/git/wft21_septum_landmark_detection/data/temp_predictions/k-fold_fold3'
ensure_dir(export_root)

df_fold = df[df['modality']=='test']
#print(df_fold)

# filter a list of filenames by a patient id, this is necessary as the filepath in our df differs from the real filenames
def filter_by_patient_id(p_id, f_names):
    return [elem for elem in f_names if p_id in elem]


#for p in df['patient'].unique(): # show only data on 'unique' patients to sum up folds and slices
for p in sorted(df_fold['patient'].unique()): # for each patient
    info(p) # shows which patient we are at
    files_ = filter_by_patient_id(p, x_val_sax)
    masks_ = filter_by_patient_id(p, y_val_sax)
    info(len(files_)) # shows amount of slices for each patient
    #print(files_[0]) # files is a list with the name of all patient data
    # collect all files for this patient
    # split in ED and ES
    ed_f = files_[:len(files_)//2]
    es_f = files_[len(files_)//2:]
    ed_m = masks_[:len(masks_)//2]
    es_m = masks_[len(masks_)//2:]
    f_ = [ed_f, es_f]
    m_ = [ed_m, es_m]
    phases = ['ED', 'ES']
    assert(len(ed_m)==len(ed_f)), 'number of images and masks should be the same, something went wrong'
    info('length of ed_f ' + str(len(ed_f)))
    info('length of es_f ' + str(len(es_f)))
    #print('this is ed_f ' + ed_f)
    #print('this is es_f ' + es_f)
    for p_ in range(2):
        phase_cmr_files = f_[p_] 
        phase_mask_files = m_[p_]
        current_phase = phases[p_]
        info('patient: {}, phase: {}, files: {}'.format(p, current_phase, len(phase_cmr_files)))
        
        validation_generator = DataGenerator(phase_cmr_files, phase_mask_files, config=val_config)

        # get cmr mask
        gts = np.stack([np.squeeze(y) for x, y in validation_generator])
        logging.info('groundtruth shape' + str(gts.shape))
        #get cmr image
        gts_cmr = np.stack([np.squeeze(x) for x, y in validation_generator])
        logging.info('original cmr shape' + str(gts_cmr.shape))

        # predict on the validation generator
        preds = model.predict(validation_generator)
        logging.info(preds.shape)

        # upper = 1, lower == 2
        # transform to int representation (one-hot)
        gts_flat = np.zeros((gts.shape[:-1]))
        gts_flat[gts[...,0]>0.5] = 1
        gts_flat[gts[...,1]>0.5] = 2

        preds_flat = np.zeros((gts.shape[:-1]))
        preds_flat[preds[...,0]>0.5] = 1
        preds_flat[preds[...,1]>0.5] = 2
        
        info(gts_flat.shape)
        info(preds_flat.shape)
        info(gts_cmr.shape)
        #print(preds_flat) #NotIncluded
        gt_sitks = sitk.GetImageFromArray(gts_flat.astype(np.uint8))
        pred_sitks = sitk.GetImageFromArray(preds_flat.astype(np.uint8))
        gt_cmr_sitks = sitk.GetImageFromArray(np.stack(gts_cmr, axis = 0))
        
        #sitk.WriteImage(sitk.GetImageFromArray(np.stack(gts_cmr[:10], axis=0)), '/mnt/ssd/git/wft21_septum_landmark_detection/data/temp/3d_new_temp_cmr.nrrd')
        
        # please write here the pred_sitk and gt_sitk and gt_cmr_sitk to disk
        sitk.WriteImage(gt_sitks, os.path.join(export_root, '{}_{}_gt.nrrd'.format(p, current_phase)))
        sitk.WriteImage(pred_sitks, os.path.join(export_root, '{}_{}_pred.nrrd'.format(p, current_phase)))
        sitk.WriteImage(gt_cmr_sitks, os.path.join(export_root, '{}_{}_cmr.nrrd'.format(p, current_phase)))
        #sitk.WriteImage(sitk.GetImageFromArray(np.stack(gts_cmr[:10], axis=0)), '/mnt/ssd/git/wft21_septum_landmark_detection/data/temp/3d_new_temp_cmr.nrrd')

logging.info('done! Check the ' + export_root + 'folder for files')

2021-07-06 16:36:24,317 INFO patient007
2021-07-06 16:36:24,318 INFO 20
2021-07-06 16:36:24,318 INFO length of ed_f 10
2021-07-06 16:36:24,318 INFO length of es_f 10
2021-07-06 16:36:24,319 INFO patient: patient007, phase: ED, files: 10
2021-07-06 16:36:24,319 INFO Create DataGenerator
2021-07-06 16:36:24,320 INFO Datagenerator created with: 
 shape: [128, 128]
 spacing: [1.8, 1.8]
 batchsize: 1
 Scaler: MinMax
 Images: 10 
 Augment: False 
 Thread workers: 8
2021-07-06 16:36:24,320 INFO No augmentation
2021-07-06 16:36:24,372 INFO groundtruth shape(10, 128, 128, 2)
2021-07-06 16:36:24,407 INFO original cmr shape(10, 128, 128)
2021-07-06 16:36:26,070 INFO (10, 128, 128, 2)
2021-07-06 16:36:26,072 INFO (10, 128, 128)
2021-07-06 16:36:26,072 INFO (10, 128, 128)
2021-07-06 16:36:26,072 INFO (10, 128, 128)
2021-07-06 16:36:26,075 INFO patient: patient007, phase: ES, files: 10
2021-07-06 16:36:26,075 INFO Create DataGenerator
2021-07-06 16:36:26,076 INFO Datagenerator created with: 
 shape:

2021-07-05 18:35:43,157 INFO patient098


## code to figure out where we are currently 

In [42]:

os.path.basename(x_val_sax[0]) #gives basename of the first file from x_val_sax
#os.path.split(x_val_sax[0])  # gives directory in which the x_val_sax files are located

'patient001__t01_z0_img.nrrd'

In [31]:
#checks_out
export_path = '/mnt/ssd/git/wft21_septum_landmark_detection/data/temp/'
ensure_dir(export_path)
_ = [sitk.WriteImage(img, '{}_{}'.format(export_path, os.path.basename(f_name))) for img, f_name in zip(gt_sitks, x_val_sax)]
# this might be important, need to figure out what it does exactly. 

# Visualizations and Helper functions

In [19]:
#checks_out
#visualizes predictions
from src.visualization.Visualize import show_2D_or_3D
@interact
def show_predictions(elem = (0,len(preds_flat)-1)):
    elem = gts_flat[elem]
    print(elem.shape)
    show_2D_or_3D(elem)

interactive(children=(IntSlider(value=237, description='elem', max=475), Output()), _dom_classes=('widget-inte…

In [21]:
#checks_out
from src.visualization.Visualize import show_2D_or_3D
@interact
def show_predictions(elem = (0, len(x_val_sax))):
    temp = x_val_sax[elem]
    cmr = sitk.ReadImage(temp)
    cmr = sitk.GetArrayFromImage(cmr)
    temp = temp.replace('img', 'msk')
    temp = sitk.ReadImage(temp)
    temp = sitk.GetArrayFromImage(temp)
    #plt.imshow(temp, cmap='gray')
    
    show_2D_or_3D(cmr, temp)

interactive(children=(IntSlider(value=238, description='elem', max=476), Output()), _dom_classes=('widget-inte…

In [22]:
#checks_out

from src.visualization.Visualize import show_2D_or_3D
@interact
def show_predictions(elem = (0,gts.shape[0])):
    temp = gts_cmr[elem]
    msk = gts_flat[elem]
    print(msk.shape)
    #plt.imshow(temp, cmap='gray')
    #plt.imshow(msk, alpha=0.8)
    show_2D_or_3D(temp, msk)

interactive(children=(IntSlider(value=238, description='elem', max=476), Output()), _dom_classes=('widget-inte…

In [23]:
#checks_out

generator = validation_generator
@interact
def select_image_in_batch(batch = (0,len(generator)-1, 1), im = (0,config['BATCHSIZE']- 1, 1), slice_n=(1,11), save=False, filepath='data/temp/', filename='temp_x.npy'):
    
    global x, y
    x, y = generator.__getitem__(batch)
    info('selected batch : ' + str(batch))
    # logging level == debug --> visualise the generator steps
    info('X shape: {}, Y shape: {}'.format(x.shape, y.shape))
    show_2D_or_3D(x[im], y[im], interpol='bilinear',dpi=100,f_size=(5,5))
    plt.show()
    
    plt.hist(x[im].flatten())
    plt.show()
    if save:
        ensure_dir(filepath)
        np.save(os.path.join(filepath, filename), x[im])
        logging.info('saved to {}'.format(os.path.join(filepath, filename)))

interactive(children=(IntSlider(value=237, description='batch', max=475), IntSlider(value=0, description='im',…

# Undo the generator steps

- If we have masks, undo the cropping/padding, resampling etc. so that our masks have the same size/spacing and name as our input volumes
- If we have regression coordinates, make sure that they could be applied on the input volumes

# Load the gts, predictions testwise

In [ ]:
# # Some sample numpy data

# data = np.zeros((5,4,3,2))
# filename = 'testdata.nrrd'

# # Write to a NRRD file
# nrrd.write(filename, data)

# # Read the data back from file
# readdata, header = nrrd.read(filename)
# print(readdata.shape)
# print(header)

## Converting numpy predictions into nrrd via pynrrd

In [ ]:
# write 3d_nrrd file to disk
sitk.WriteImage(sitk.GetImageFromArray(np.stack(gts_cmr[:10], axis=0)), '/mnt/ssd/git/wft21_septum_landmark_detection/data/temp/3d_temp_cmr.nrrd')

In [ ]:
# write 3d_nrrd file to disk
sitk.WriteImage(sitk.GetImageFromArray(np.stack(gts_cmr[:10], axis=0)), '/mnt/ssd/git/wft21_septum_landmark_detection/data/temp/3d_temp_cmr.nrrd')